<a href="https://colab.research.google.com/github/psychemistz/MultiDimGCNR/blob/main/NMB_TCB_Image_Denoising_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install wget

import os
from os import path
import wget
import h5py
import numpy as np
from scipy.io import savemat

from keras.utils.np_utils import to_categorical
from sklearn.feature_extraction import image

In [18]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras.models import Model
from keras.models import load_model
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

In [6]:
## Download Datasets from github
classes = ('cloudy','rain','shine','sunrise')
data_path = 'https://raw.githubusercontent.com/Shujaat123/Weather_Classification/master/dataset/'

flist = []
for fname in classes:
  filename = 'WeatherClassificationDB_'+fname+'.mat'
  if(path.exists(filename)):
    !rm $filename
    print('existing file:', filename, ' has been deleted')
  print('downloading latest version of file:', filename)
  file_path = data_path + filename
  wget.download(file_path, filename)
  print('DONE')
  flist.append(filename)

downloading latest version of file: WeatherClassificationDB_cloudy.mat
DONE
downloading latest version of file: WeatherClassificationDB_rain.mat
DONE
downloading latest version of file: WeatherClassificationDB_shine.mat
DONE
downloading latest version of file: WeatherClassificationDB_sunrise.mat
DONE


In [8]:
## Load Datasets into Env
cloudy_imgs = h5py.File(flist[0], 'r')['images']['input']
cloudy_labels = h5py.File(flist[0], 'r')['images']['label']

rain_imgs = h5py.File(flist[1], 'r')['images']['input']
rain_labels = h5py.File(flist[1], 'r')['images']['label']

shine_imgs = h5py.File(flist[2], 'r')['images']['input']
shine_labels = h5py.File(flist[2], 'r')['images']['label']

sunrise_imgs = h5py.File(flist[3], 'r')['images']['input']
sunrise_labels = h5py.File(flist[3], 'r')['images']['label']

In [12]:
## Check Image Dimensions
print(cloudy_imgs.shape) # (NHWC)
print(cloudy_labels.shape)

print(rain_imgs.shape) # (NHWC)
print(rain_labels.shape)

print(shine_imgs.shape) # (NHWC)
print(shine_labels.shape)

print(sunrise_imgs.shape) # (NHWC)
print(sunrise_labels.shape)


InputImages = np.concatenate((cloudy_imgs,rain_imgs,shine_imgs,sunrise_imgs), axis = 0)
InputImages = InputImages/InputImages.max()
ClassLabels = np.concatenate((cloudy_labels,rain_labels,shine_labels,sunrise_labels), axis = 0)

InputImages.shape
ClassLabels.shape
ClassLabels = to_categorical(np.squeeze(ClassLabels)-1)
ClassLabels.shape

(207, 256, 256, 3)
(207, 1, 1, 1)
(215, 256, 256, 3)
(215, 1, 1, 1)
(253, 256, 256, 3)
(253, 1, 1, 1)
(357, 256, 256, 3)
(357, 1, 1, 1)


(1032, 4)

In [13]:
## Check Sample Statistics
cloudy_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==0)).flatten())
rain_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==1)).flatten())
shine_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==2)).flatten())
sunrise_list = list(np.asarray(np.where(ClassLabels.argmax(axis=1)==3)).flatten())
total_list = cloudy_list + rain_list + shine_list + sunrise_list

print('Number of \'cloudy\' samples:',len(cloudy_list))
print('Number of \'rain\' samples:',len(rain_list))
print('Number of \'shine\' samples:',len(shine_list))
print('Number of \'sunrise\' samples:',len(sunrise_list))
print('Total number of samples:',len(total_list))

Number of 'cloudy' samples: 207
Number of 'rain' samples: 215
Number of 'shine' samples: 253
Number of 'sunrise' samples: 357
Total number of samples: 1032


In [14]:
## Train Test Split
cloudy_train = cloudy_list[0:150]
rain_train = rain_list[0:150]
shine_train = shine_list[0:150]
sunrise_train = sunrise_list[0:150]

train_list = cloudy_train + rain_train + shine_train + sunrise_train

Input_train = InputImages[train_list]
Label_train = ClassLabels[train_list]

# valid select 20 cloudy, 20 rain, 20 shine and 20 sunrise samples
cloudy_val = cloudy_list[150:170]
rain_val = rain_list[150:170]
shine_val = shine_list[150:170]
sunrise_val = sunrise_list[150:170]
val_list = cloudy_val + rain_val + shine_val + sunrise_val

Input_val = InputImages[val_list]
Label_val = ClassLabels[val_list]

## test
test_list = list(set(total_list) - set(train_list) - set(val_list))

# test_list
Input_test = InputImages[test_list]
Label_test = ClassLabels[test_list]

In [17]:
## Check Adding Gaussian Noise function
print(Input_train.shape)
temp = np.empty((1,64,64,3))
# temp2 = np.empty((1,64,64,3))
for i in range(0, Input_train.shape[0]):
  patches = image.extract_patches_2d(Input_train[i], (64, 64), random_state=40, max_patches=16)
  # noisy = patches + 0.175 * np.random.normal(0, 1, size= patches.shape)
  temp = np.concatenate((temp,patches), axis = 0)
  # temp2 = np.concatenate((temp2,noisy), axis = 0)
y_train = np.asarray(temp[1:])

x_train = y_train + 0.1 * np.random.normal(0, 1, size= y_train.shape)
x_train = x_train/np.max(np.abs(x_train))
x_train = np.clip(x_train, 0., 1.)
print(x_train.shape, y_train.shape)

y_val = Input_val
x_val = y_val + 0.1 * np.random.normal(0, 1, size= y_val.shape)
x_val = x_val/np.max(np.abs(x_val))
x_val = np.clip(x_val, 0., 1.)

print(y_val.shape, x_val.shape)

y_test = Input_test
x_test = y_test + 0.1 * np.random.normal(0, 1, size= y_test.shape)
x_test = x_test/np.max(np.abs(x_test))
x_test = np.clip(x_test, 0., 1.)
print(y_test.shape, y_test.shape)

(600, 256, 256, 3)
(9600, 64, 64, 3) (9600, 64, 64, 3)
(80, 256, 256, 3) (80, 256, 256, 3)
(352, 256, 256, 3) (352, 256, 256, 3)


In [19]:
## Define data loading function
def load_data(Input_train, Input_val, Input_test):
  temp = np.empty((1,64,64,3))

  for i in range(0, Input_train.shape[0]):
    patches = image.extract_patches_2d(Input_train[i], (64, 64), random_state=40, max_patches=16)
    temp = np.concatenate((temp,patches), axis = 0)
  y_train = np.asarray(temp[1:])

  x_train = y_train + 0.1 * np.random.normal(0, 1, size= y_train.shape)
  x_train = x_train/np.max(np.abs(x_train))
  x_train = np.clip(x_train, 0., 1.)
  # print(x_train.shape, y_train.shape)


  y_val = Input_val
  x_val = y_val + 0.1 * np.random.normal(0, 1, size= y_val.shape)
  x_val = x_val/np.max(np.abs(x_val))
  x_val = np.clip(x_val, 0., 1.)

  # print(y_val.shape, x_val.shape)

  y_test = Input_test
  x_test = y_test + 0.1 * np.random.normal(0, 1, size= y_test.shape)
  x_test = x_test/np.max(np.abs(x_test))
  x_test = np.clip(x_test, 0., 1.)
  # print(y_test.shape, y_test.shape)
  return x_train, y_train, x_val, y_val, x_test, y_test

In [21]:
## Define NMB_TCB Model

def RB(input_rb1, num_filter = 64, kernel_size = 3):
  x = Conv2D(num_filter, kernel_size, padding='same')(input_rb1)
  x = BatchNormalization()(x)
  x = Activation(activation='relu')(x)
  x = Conv2D(num_filter, kernel_size, padding='same')(x)
  x = BatchNormalization()(x)
  x = Add()([input_rb1, x])
  y = Activation(activation='relu')(x)
  return y

def MDRB(MDRB_input, num_filter = 64, kernel_size = 3, D1 = (1,1), D2 = (2,2), D3 = (3,3)):
  x = Conv2D(num_filter, kernel_size, padding = 'same')(MDRB_input)
  x = BatchNormalization()(x)
  x = Activation(activation='relu')(x)
  xd1_1 = Conv2D(num_filter, kernel_size, padding = 'same', dilation_rate= D1)(x)
  xd2_1 = Conv2D(num_filter, kernel_size, padding = 'same', dilation_rate=D2)(x)
  xd3_1 = Conv2D(num_filter, kernel_size, padding = 'same', dilation_rate=D3)(x)

  xd1_2 = Conv2D(num_filter, kernel_size, padding = 'same', dilation_rate=D1)(xd1_1)
  xd2_2 = Conv2D(num_filter, kernel_size, padding = 'same', dilation_rate=D2)(xd2_1)
  xd3_2 = Conv2D(num_filter, kernel_size, padding = 'same', dilation_rate=D3)(xd3_1)

  x = Concatenate()([xd1_2, xd2_2, xd3_2])
  x = Conv2D(num_filter, (1,1), padding = 'same')(x)
  x = BatchNormalization()(x)
  x = Add()([x, MDRB_input])
  y = Activation(activation='relu')(x)
  return y

def NMB(input_img, num_filter = 64, kernel_size = 3, num_RB = 8):
  x = Conv2D(num_filter, kernel_size, padding='same')(input_img)
  for i in range(0, num_RB):
    x = RB(x, num_filter, kernel_size)
  y = Conv2D(input_img.shape[3], kernel_size, padding='same')(x)
  return y

def TCB(input_img, output_nmb, num_filter = 64, kernel_size = 3, D1 = (1,1), D2 = (2,2), D3 = (3,3), num_MDRB = 8):
  x = Concatenate()([input_img, output_nmb])
  x = Conv2D(num_filter, kernel_size, padding = 'same')(x)
  for i in range(0, num_MDRB):
    x - MDRB(x, num_filter, kernel_size, D1, D2, D3)
  y = Conv2D(input_img.shape[3], kernel_size, padding = 'same')(x)
  return y

def NMB_TCB():
  num_filter = 64
  kernel_size = 3
  strides = 1
  num_RB = 8
  D1 = (1,1)
  D2 = (2,2)
  D3 = (3,3)
  num_MDRB = 8
  lambda_1 = 0.1
  lambda_2 = 0.9
  
  # input_img = Input(shape = (256, 256, 3), name='input_img')
  input_img = Input(shape = (None, None, 3), name='input_img')
  
  output_NMB = NMB(input_img, num_filter, kernel_size, num_RB)
  output_NMB = Add(name = 'output_NMB')([input_img, output_NMB])

  output_TCB = TCB(input_img, output_NMB, num_filter, kernel_size, D1 , D2, D3, num_MDRB)
  
  output_CB = Add(name = 'output_CB')([output_NMB, output_TCB])

  model = Model(inputs=[input_img], outputs=[output_CB, output_NMB]) 
  model.compile(optimizer=Adam(learning_rate = 1e-4, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08),
                  loss={ 'output_CB': 'mean_squared_error', 'output_NMB': 'mean_squared_error'},
                  loss_weights={'output_NMB': lambda_1, 'output_CB': lambda_2}
                )

  return model


model = NMB_TCB()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   1792        ['input_img[0][0]']              
                                64)                                                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, None, None,   36928       ['conv2d[0][0]']                 
                                64)                                                           

In [ ]:
num_trials = 3

if not os.path.exists("models"):
   os.mkdir("models")

DATA_PATH = "./results"

for loop_ind in range(0, num_trials):
  x_train, y_train, x_val, y_val, x_test, y_test = load_data(Input_train, Input_val, Input_test)
  
  model = NMB_TCB()
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  
  checkpoint = ModelCheckpoint('./models\\model-best.h5',
                                    verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
  history = model.fit(x_train, y_train,
                        validation_data = (x_val, y_val),
                        epochs=200, batch_size=64, callbacks=[checkpoint, es], verbose=1)
  
  del model
  model = keras.models.load_model('./models\\model-best.h5')

  test_pred, a = model.predict(x_test)
  test_pred = np.clip(test_pred, 0., 1.)
  
  save_path = DATA_PATH + '/TCB_NMB_200_epochs/trial_' + str(loop_ind)
  if (not(os.path.exists(save_path))):
    os.makedirs(save_path)
  save_path_target = save_path + '/Target.mat'
  save_path_input = save_path + '/Input.mat'
  save_path_pred = save_path + '/Predicted.mat'
  # print(save_path)
  savemat(save_path_target, {'Target':y_test})
  savemat(save_path_target, {'Input':x_test})
  savemat(save_path_predicted, {'Predicted':test_pred})


Epoch 1: val_loss improved from inf to 0.01544, saving model to ./models\model-best.h5

Epoch 2: val_loss improved from 0.01544 to 0.00988, saving model to ./models\model-best.h5

Epoch 3: val_loss improved from 0.00988 to 0.00988, saving model to ./models\model-best.h5

Epoch 4: val_loss did not improve from 0.00988

Epoch 5: val_loss improved from 0.00988 to 0.00854, saving model to ./models\model-best.h5

Epoch 6: val_loss improved from 0.00854 to 0.00741, saving model to ./models\model-best.h5
